## Initiate the Toolbox
- the parameters set here will be used to collect command-specific data when the `Fetcher` executes the TET pattern.

In [1]:
%load_ext autoreload
%autoreload

from humbldata.toolbox.toolbox_controller import Toolbox

toolbox = Toolbox(
    symbols=["DKNG", "HII", "XYL", "GTBIF", "EAT", "MPW"],
    interval="1d",
    start_date="2000-01-01",
    end_date="2024-02-26",
    membership="admin"
)
toolbox

Toolbox(symbols=['DKNG', 'HII', 'XYL', 'GTBIF', 'EAT', 'MPW'], interval=1d, start_date=2000-01-01, end_date=2024-02-26, provider=yfinance, membership=admin)

## Run Mandelbrot Channel Calculation

In [2]:
# import nest_asyncio
# nest_asyncio.apply()

result = toolbox.technical.mandelbrot_channel(historical=False, window="3m", _boundary_group_down=False, chart=True)
# result.to_polars()

INFO: MandelbrotChannelFetcher || START: fetch_data (sync)
INFO: MandelbrotChannelFetcher || END: fetch_data (sync) - Total time: 1.8580s


In [3]:
json_result_chart = result.to_json(chart=True)

In [4]:
json_result_chart

['{"data":[{"line":{"color":"blue"},"name":"Recent Price","x":["2011-10-13","2011-10-14","2011-10-17","2011-10-18","2011-10-19","2011-10-20","2011-10-21","2011-10-24","2011-10-25","2011-10-26","2011-10-27","2011-10-28","2011-10-31","2011-11-01","2011-11-02","2011-11-03","2011-11-04","2011-11-07","2011-11-08","2011-11-09","2011-11-10","2011-11-11","2011-11-14","2011-11-15","2011-11-16","2011-11-17","2011-11-18","2011-11-21","2011-11-22","2011-11-23","2011-11-25","2011-11-28","2011-11-29","2011-11-30","2011-12-01","2011-12-02","2011-12-05","2011-12-06","2011-12-07","2011-12-08","2011-12-09","2011-12-12","2011-12-13","2011-12-14","2011-12-15","2011-12-16","2011-12-19","2011-12-20","2011-12-21","2011-12-22","2011-12-23","2011-12-27","2011-12-28","2011-12-29","2011-12-30","2012-01-03","2012-01-04","2012-01-05","2012-01-06","2012-01-09","2012-01-10","2012-01-11","2012-01-12","2012-01-13","2012-01-17","2012-01-18","2012-01-19","2012-01-20","2012-01-23","2012-01-24","2012-01-25","2012-01-26","

In [10]:
json_result_chart[0].__class__

str

In [9]:
    import json
    import pytest

    for chart in json_result_chart:
        try:
            json.loads(chart)
        except json.JSONDecodeError:
            pytest.fail(f"Chart data result is not valid JSON: {chart}")

In [21]:
if isinstance(result.chart, list):
    chart_content = [
        chart.content
        for chart in result.chart
        if chart and chart.content
    ]

In [10]:
isinstance(chart_content, list)

True

In [32]:
data = result.to_polars()
data

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2024-02-26,"""DKNG""",34.4745,40.349998,43.355
2024-02-26,"""EAT""",43.5146,46.200001,51.0035
2024-02-26,"""GTBIF""",10.8649,13.65,15.6322
2024-02-26,"""HII""",266.0752,286.837952,298.8332
2024-02-26,"""MPW""",3.4321,3.620531,3.7639
2024-02-26,"""XYL""",120.1684,125.381935,128.1258


In [6]:
from humbldata.toolbox.technical.mandelbrot_channel.model import calc_mandelbrot_channel_historical_mp, calc_mandelbrot_channel_historical_concurrent


# result_mp = calc_mandelbrot_channel_historical_mp(
#     data=data,
#     window="1m",
#     rv_adjustment=True,
#     rv_method="std",
#     rs_method="RS",
#     rv_grouped_mean=False,
#     live_price=False
# )
result_cc = calc_mandelbrot_channel_historical_concurrent(
    data=data,
    window="1m",
    rv_adjustment=True,
    rv_method="std",
    rs_method="RS",
    rv_grouped_mean=False,
    live_price=False,
    use_processes=False,
    max_workers=16
)

In [7]:
# result_mp.collect()
result_cc.collect()

date,symbol,bottom_price,recent_price,top_price
date,str,f64,f64,f64
2023-02-02,"""AAPL""",143.3908,149.587631,152.2181
2023-02-03,"""AAPL""",146.139,153.237579,154.5305
2023-02-06,"""AAPL""",145.8606,150.490189,156.4026
2023-02-07,"""AAPL""",147.7807,153.386353,156.5727
2023-02-08,"""AAPL""",147.7869,150.67865,158.0685
…,…,…,…,…
2023-12-22,"""TTEK""",158.1424,165.538162,168.7606
2023-12-26,"""TTEK""",158.6096,166.256165,169.5257
2023-12-27,"""TTEK""",158.083,167.921524,167.9215
